# The Ashrae project

> Building models for the Ashrae prediction challenge.

## TODO
* continue debugging `preprocessing.ipynb`: high resource demand with `nbdev_test_nbs`, figure out why and fix

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from ashrae import loading, preprocessing, feature_testing, leaderboard, inspection, modelling
import plotly.express as px
from fastcore.foundation import L, Path
from fastai.tabular.all import *

In [ ]:
#hide
pd.options.plotting.backend = "plotly"

## Configuring

Defining wether to process the test set (warning, this alone takes 12+ minutes) and submit the results to kaggel (you will need your credentials set up).

In [ ]:
do_test = True
do_submit = False

Defining where the csv files are located

In [ ]:
loading.DATA_PATH = Path("../data")

## Getting the data from Kaggle

In [ ]:
# !kaggle competitions download -c ashrae-energy-prediction -p {data_path}

In [ ]:
# !unzip {data_path}/ashrae-energy-prediction.zip -d {data_path}/

In [ ]:
# !kaggle competitions leaderboard -c ashrae-energy-prediction -p {data_path} --download

In [ ]:
# !unzip {data_path}/ashrae-energy-prediction.zip -d {data_path}/

## Loading

In [ ]:
%%time
loading.N_TRAIN = 10_000
loading.N_TEST = 10_000

In [ ]:
%%time
csvs = loading.get_csvs()
csvs

In [ ]:
%%time
ashrae_data = loading.load_all()

## Inspecting the leaderboard

In [ ]:
df_leaderboard = pd.read_csv(csvs['public-leaderboard'], parse_dates=['SubmissionDate'])
df_leaderboard.head()

In [ ]:
%%time
dis = leaderboard.get_leaderboard_distribution(df_leaderboard)
dis['Score'].describe(percentiles=[.05, .1, .25, .5, .75, .95])

## Building features

In [ ]:
%%time
processor = preprocessing.Processor() 
tfms_config = {
    'add_time_features':{},
    'add_weather_features':{'fix_time_offset':True,
                            'add_na_indicators':True,
                            'impute_nas':True},
    'add_building_features':{},
}

df, var_names = processor(ashrae_data['meter_train'], tfms_configs=tfms_config,
                          df_weather=ashrae_data['weather_train'],
                          df_building=ashrae_data['building'])

if do_test:
    %time
    df_test, _ = processor(ashrae_data['meter_test'], tfms_configs=tfms_config,
                             df_weather=ashrae_data['weather_test'],
                             df_building=ashrae_data['building'])
    df_test = preprocessing.align_test(df, var_names, df_test)

## Sampling from `df`

In [ ]:
%%time
n = len(df)

if False: # per building_id and meter sampling
    n_sample_per_bid = 500
    replace = True

    df = (df.groupby(['building_id', 'meter'])
         .sample(n=n_sample_per_bid, replace=replace))

if False: # general sampling
    frac_samples = .05
    replace = False

    df = (df.sample(frac=frac_samples, replace=replace))

print(f'using {len(df)} samples = {len(df)/n*100:.2f} %')

## Preparing the data for modelling

In [ ]:
%%time
# t_train = pd.read_parquet(data_path/'t_train.parquet')
t_train = None

%time
#split_kind = 'random'
#split_kind = 'time'
# split_kind = 'fix_time'
split_kind = 'time_split_day'

In [ ]:
%%time
train_frac = .9
splits = preprocessing.split_dataset(df, split_kind=split_kind, train_frac=train_frac,
                                     t_train=t_train)
print(f'sets {len(splits)}, train {len(splits[0])} = {len(splits[0])/len(df):.4f}, valid {len(splits[1])} = {len(splits[1])/len(df):.4f}')

In [ ]:
%%time
# procs = [] 
procs = [Categorify, FillMissing, Normalize]
to = feature_testing.get_tabular_object(df,
                                        var_names,
                                        splits=splits,
                                        procs=procs)

In [ ]:
%%time
train_bs = 256
val_bs = 256

dls = to.dataloaders(bs=train_bs, val_bs=val_bs)

In [ ]:
%%time
test_bs = 1024

if do_test:
    test_dl = dls.test_dl(df_test, bs=test_bs) 

## Training a neural net using `tabular_learner`

In [ ]:
layers = [50, 25, 12]
y_range = (-.1, 17)

config = tabular_config(embed_p=.01, ps = [.5, .5, .5])

learn = tabular_learner(dls, y_range=y_range, layers=layers,
                        n_out=1, config=config, 
                        loss_func=modelling.evaluate_torch)

In [ ]:
learn.lr_find()

In [ ]:
%%time
learn.fit_one_cycle(5, lr_max=1e-2)

In [ ]:
learn.recorder.plot_loss()

## Inspecting the predictions

### Basic score

In [ ]:
%%time
y_valid_pred, y_valid_true = learn.get_preds()

In [ ]:
%%time
if do_test:
    y_test_pred, _ = learn.get_preds(dl=test_dl)
    y_test_pred = modelling.cnr(y_test_pred)

In [ ]:
nb_score = modelling.evaluate_torch(torch.tensor(y_valid_true), 
                                    torch.tensor(y_valid_pred)).item()
print(f'fastai loss {nb_score:.4f}')

### Histogram of  `dep_var`

In [ ]:
feature_testing.hist_plot_preds(modelling.pick_random(y_valid_true), 
                                modelling.pick_random(y_valid_pred), 
                                label0='truth', label1='prediction')

In [ ]:
if do_test:
    feature_testing.hist_plot_preds(modelling.pick_random(y_valid_true), 
                                    modelling.pick_random(y_test_pred), 
                                    label0='truth (validation)', 
                                    label1='prediction (test set)').show()

### Confidently wrong predictions by `building_id`

In [ ]:
%%time
miss_cols = [v for v in ['building_id', 'meter','timestamp'] if v not in to.valid.xs.columns]
tmp = to.valid.xs.join(df.loc[:,miss_cols]) if len(miss_cols)>0 else to.valid.xs
bwt = feature_testing.BoldlyWrongTimeseries(tmp, y_valid_true, y_valid_pred)

In [ ]:
bwt.run_boldly()

## Submission

In [ ]:
%%time
if do_test and do_submit:
    y_test_pred_original = torch.exp(tensor(y_test_pred)) - 1

    y_out = pd.DataFrame(cnr(y_test_pred_original),
                         columns=['meter_reading'],
                         index=df_test.index)
    display(y_out.head())

    assert len(y_out) == 41697600

In [ ]:
%%time
if do_submit:
    y_out.to_csv(data_path/'my_submission.csv')

In [ ]:
# message = ['random forest', '500 obs/bid', 'all features', f'nb score {nb_score:.4f}']
message = ['lightgbm', '500 obs/bid', '100 rounds', '42 leaves', 'lr .5', f'nb score {nb_score:.4f}']
# message = ['tabular_learner', '500 obs/bid', 'all features', f'layers {layers}, embed_p .1, ps [.1,.1,.1]', f'nb score {nb_score:.4f}']
message = ' + '.join(message)
message

In [ ]:
if do_test and do_submit:
    print('Submitting...')
    !kaggle competitions submit -c ashrae-energy-prediction -f '{data_path}/my_submission.csv' -m '{message}'